In [37]:
import pandas as pd
import numpy as np

In [5]:
books = pd.read_csv('all_books.csv', index_col=0)

In [6]:
books

,author,title,text
0,austen,Emma-JA,"['volume', 'chapter', 'emma', 'woodhouse', 'ha..."
1,austen,LS-JA,"['lady', 'susan', 'vernon', 'mr', 'vernon', 'l..."
2,austen,Mansfield-JA,"['chapter', 'thirty', 'year', 'ago', 'miss', '..."
3,austen,Northranger_Abbey-JA,"['advertisement', 'authoress', 'northanger', '..."
4,austen,PP_JA,"['chapter', '1', 'truth', 'universally', 'ackn..."
5,austen,Persuasion-JA,"['chapter', '1', 'sir', 'walter', 'elliot', 'k..."
6,austen,SS-JA,"['chapter', '1', 'family', 'dashwood', 'long',..."
7,dickens,dickens-american-631,"['chapter', 'going', 'away', 'shall', 'never',..."
8,dickens,dickens-battle-630,"['chapter', 'part', 'first', 'upon', 'time', '..."
9,dickens,dickens-childs-629-copy,"['chapter', 'ancient', 'england', 'roman', 'lo..."


In [7]:
books.author.unique()

array(['austen', 'dickens', 'doyle', 'emerson', 'hawthorne', 'keats',
       'melville', 'milton', 'poe', 'shakespeare', 'twain', 'wilde'],
      dtype=object)

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [10]:
bookvect = CountVectorizer()

In [14]:
# count vectorize all the text of books
X_train_counts = bookvect.fit_transform(books['text'])

In [17]:
X_train_counts

<193x104556 sparse matrix of type '<class 'numpy.int64'>'
	with 683367 stored elements in Compressed Sparse Row format>

In [18]:
tdif = TfidfVectorizer()

In [22]:
tdif_matrix = tdif.fit_transform(books['text'])

In [23]:
tdif_matrix

<193x104556 sparse matrix of type '<class 'numpy.float64'>'
	with 683367 stored elements in Compressed Sparse Row format>

In [38]:
target = books.author

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_train_counts, target, stratify=target)

In [40]:
clf = GaussianNB()
clf.fit(X_train.todense(), y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [43]:
preds = clf.predict(X_test.todense())

In [51]:
accuracy = accuracy_score(y_test, preds)

#f1 = f1_score(y_test, preds, average=None)

print("Accuracy Score for model: {:.4}%".format(accuracy * 100))
#print("F1 Score for model: {:.4}%".format(f1 * 100))

Accuracy Score for model: 65.31%
